In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import _pickle as cPickle
import numpy as np
import numpy



In [3]:
subject_collection = {} 
for i in range(1,33):
    if (i<10):
        with open('raw/s0'+str(i)+'.dat', 'rb') as f:
            x= cPickle.load(f, encoding='latin1')
            subject_collection[i-1]=x
    else:
        with open('raw/s'+str(i)+'.dat', 'rb') as f:
            x= cPickle.load(f, encoding='latin1')
            subject_collection[i-1]=x

In [4]:
xjk=subject_collection[0]
xjkX=xjk["data"]
np.shape(xjkX)

(40, 40, 8064)

In [6]:
xjkY=xjk["labels"]
np.shape(xjkY)

(40, 4)

In [12]:
fulldata={}
fulllabel={}
for i in range(32):
    x=subject_collection[i]
    X = x["data"]
    Y=x["labels"]
    fulldata[i]=X
    fulllabel[i]=Y


    

In [6]:
#here we are naming the columns for the eeg data that we will tanspose and we will name the columns 
sen_list = [("sen"+str(i)) for i in range(40)]

In [9]:
def dfa(X, Ave=None, L=None):
    """Compute Detrended Fluctuation Analysis from a time series X and length of
    boxes L.

    The first step to compute DFA is to integrate the signal. Let original
    series be X= [x(1), x(2), ..., x(N)].

    The integrated signal Y = [y(1), y(2), ..., y(N)] is obtained as follows
    y(k) = \sum_{i=1}^{k}{x(i)-Ave} where Ave is the mean of X.

    The second step is to partition/slice/segment the integrated sequence Y
    into boxes. At least two boxes are needed for computing DFA. Box sizes are
    specified by the L argument of this function. By default, it is from 1/5 of
    signal length to one (x-5)-th of the signal length, where x is the nearest
    power of 2 from the length of the signal, i.e., 1/16, 1/32, 1/64, 1/128,
    ...

    In each box, a linear least square fitting is employed on data in the box.
    Denote the series on fitted line as Yn. Its k-th elements, yn(k),
    corresponds to y(k).

    For fitting in each box, there is a residue, the sum of squares of all
    offsets, difference between actual points and points on fitted line.

    F(n) denotes the square root of average total residue in all boxes when box
    length is n, thus
    Total_Residue = \sum_{k=1}^{N}{(y(k)-yn(k))}
    F(n) = \sqrt(Total_Residue/N)

    The computing to F(n) is carried out for every box length n. Therefore, a
    relationship between n and F(n) can be obtained. In general, F(n) increases
    when n increases.

    Finally, the relationship between F(n) and n is analyzed. A least square
    fitting is performed between log(F(n)) and log(n). The slope of the fitting
    line is the DFA value, denoted as Alpha. To white noise, Alpha should be
    0.5. Higher level of signal complexity is related to higher Alpha.

    Parameters
    ----------

    X:
        1-D Python list or numpy array
        a time series

    Ave:
        integer, optional
        The average value of the time series

    L:
        1-D Python list of integers
        A list of box size, integers in ascending order

    Returns
    -------

    Alpha:
        integer
        the result of DFA analysis, thus the slope of fitting line of log(F(n))
        vs. log(n). where n is the

    Examples
    --------
    >>> import pyeeg
    >>> from numpy.random import randn
    >>> print(pyeeg.dfa(randn(4096)))
    0.490035110345

    Reference
    ---------
    Peng C-K, Havlin S, Stanley HE, Goldberger AL. Quantification of scaling
    exponents and crossover phenomena in nonstationary heartbeat time series.
    _Chaos_ 1995;5:82-87

    Notes
    -----

    This value depends on the box sizes very much. When the input is a white
    noise, this value should be 0.5. But, some choices on box sizes can lead to
    the value lower or higher than 0.5, e.g. 0.38 or 0.58.

    Based on many test, I set the box sizes from 1/5 of    signal length to one
    (x-5)-th of the signal length, where x is the nearest power of 2 from the
    length of the signal, i.e., 1/16, 1/32, 1/64, 1/128, ...

    You may generate a list of box sizes and pass in such a list as a
    parameter.

    """

    X = numpy.array(X)

    if Ave is None:
        Ave = numpy.mean(X)

    Y = numpy.cumsum(X)
    Y -= Ave

    if L is None:
        L = numpy.floor(len(X) * 1 / (
            2 ** numpy.array(list(range(4, int(numpy.log2(len(X))) - 4))))
        )

    F = numpy.zeros(len(L))  # F(n) of different given box length n

    for i in range(0, len(L)):
        n = int(L[i])                        # for each box length L[i]
        if n == 0:
            print("time series is too short while the box length is too big")
            print("abort")
            exit()
        for j in range(0, len(X), n):  # for each box
            if j + n < len(X):
                c = list(range(j, j + n))
                # coordinates of time in the box
                c = numpy.vstack([c, numpy.ones(n)]).T
                # the value of data in the box
                y = Y[j:j + n]
                # add residue in this box
                F[i] += numpy.linalg.lstsq(c, y)[1]
        F[i] /= ((len(X) / n) * n)
    F = numpy.sqrt(F)

    Alpha = numpy.linalg.lstsq(numpy.vstack(
        [numpy.log(L), numpy.ones(len(L))]
    ).T, numpy.log(F))[0][0]

    return Alpha

In [10]:
#getting the Pfd feature dataset 
for i in range(32):
    X=fulldata[i]
    dataframe_collection = {}
    #creating 40 dataframe for each subject
    for j in range(40):
        d=pd.DataFrame(X[j])
        dt=d.transpose()
        dt.columns=sen_list
        dataframe_collection[j] = dt
    #after running this loop dataset list will have 40 rows 
    dataset=[]
    for j in range(40):
        row=[]
        l=dataframe_collection[j]
        #row list will have 40 hurst feature for 40 sensors creating 1 row for the dataset
        for k in range(40):
            h=dfa(l[l.columns[k]])
            row.append(h)
        dataset.append(row)
    #now we will make a dataframe from the the dataset and assign it a column name 
    g=pd.DataFrame(dataset)
    Hurst_list = [("senDfa"+str(lg)) for lg in range(40)]
    g.columns=Hurst_list
    #as computing the features take a lot of time we save them to csv files :D 
    g.to_csv("features/s"+str(i)+"Dfa.csv", index=False)
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:120: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [6]:
from __future__ import print_function
import numpy


# ####################### Begin function definitions #######################

def hurst(X):
    """ Compute the Hurst exponent of X. If the output H=0.5,the behavior
    of the time-series is similar to random walk. If H<0.5, the time-series
    cover less "distance" than a random walk, vice verse.

    Parameters
    ----------

    X

        list

        a time series

    Returns
    -------
    H

        float

        Hurst exponent

    Notes
    --------
    Author of this function is Xin Liu

    Examples
    --------

    >>> import pyeeg
    >>> from numpy.random import randn
    >>> a = randn(4096)
    >>> pyeeg.hurst(a)
    0.5057444

    """
    X = numpy.array(X)
    N = X.size
    T = numpy.arange(1, N + 1)
    Y = numpy.cumsum(X)
    Ave_T = Y / T

    S_T = numpy.zeros(N)
    R_T = numpy.zeros(N)

    for i in range(N):
        S_T[i] = numpy.std(X[:i + 1])
        X_T = Y - T * Ave_T[i]
        R_T[i] = numpy.ptp(X_T[:i + 1])

    R_S = R_T / S_T
    R_S = numpy.log(R_S)[1:]
    n = numpy.log(T)[1:]
    A = numpy.column_stack((n, numpy.ones(n.size)))
    [m, c] = numpy.linalg.lstsq(A, R_S)[0]
    H = m
    return H

In [16]:
#getting the hurst data 
for i in range(32):
    X=fulldata[i]
    dataframe_collection = {}
    #creating 40 dataframe for each subject
    for j in range(40):
        d=pd.DataFrame(X[j])
        dt=d.transpose()
        dt.columns=sen_list
        dataframe_collection[j] = dt
    #after running this loop dataset list will have 40 rows 
    dataset=[]
    for j in range(40):
        row=[]
        l=dataframe_collection[j]
        #row list will have 40 hurst feature for 40 sensors creating 1 row for the dataset
        for k in range(40):
            h=hurst(l[l.columns[k]])
            row.append(h)
        dataset.append(row)
    #now we will make a dataframe from the the dataset and assign it a column name 
    g=pd.DataFrame(dataset)
    Hurst_list = [("senHurst"+str(lg)) for lg in range(40)]
    g.columns=Hurst_list
    #now to complete the dataset we also add the labels
    dfx=pd.DataFrame(Y)
    g.join(dfx)
    g.head()
    #as computing the features take a lot of time we save them to csv files :D 
    g.to_csv("features/s"+str(i)+"Hurst.csv", index=False)
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [7]:
yu=fulllabel[0]
gy=pd.DataFrame(yu)
Label_list = ["Valance","Arousal","Dominance","Liking"]
gy.columns=Label_list
gy

Valance  Arousal  Dominance  Liking
0      7.71     7.60       6.90    7.83
1      8.10     7.31       7.28    8.47
2      8.58     7.54       9.00    7.08
3      4.94     6.01       6.12    8.06
4      6.96     3.92       7.19    6.05
5      8.27     3.92       7.00    8.03
6      7.44     3.73       7.08    7.04
7      7.32     2.55       6.32    5.87
8      4.04     3.29       3.62    5.99
9      1.99     4.86       2.04    7.09
10     2.99     2.36       3.63    6.24
11     2.71     2.77       3.40    7.35
12     1.95     3.12       2.87    6.18
13     4.18     2.24       3.04    5.04
14     3.17     8.08       2.91    5.04
15     6.81     7.44       8.15    7.14
16     2.46     6.91       6.77    6.41
17     7.23     7.15       6.94    8.01
18     7.17     8.00       8.10    6.79
19     8.26     7.91       7.19    8.13
20     9.00     7.95       8.37    7.86
21     7.09     2.08       7.06    7.37
22     8.15     3.01       7.37    7.90
23     7.04     7.09       8.01    8.22
24     8.86     7.21       8.65    7.21
25     7.28     7.27       7.41    8.24
26     7.35     6.95       7.03    7.29
27     3.88     3.35       4.01    7.87
28     1.36     2.27       3.00    8.14
29     2.08     2.99       3.22    7.33
30     3.03     8.14       2.86    8.04
31     2.28     8.00       3.27    3.95
32     3.81     3.85       4.78    5.13
33     2.28     7.09       7.28    6.92
34     2.06     8.15       8.05    5.18
35     2.90     6.92       6.50    3.87
36     2.31     6.88       3.10    6.77
37     3.33     7.18       6.54    6.62
38     3.24     6.18       7.87    6.15
39     5.10     7.12       6.17    5.97

In [8]:
gg=pd.read_csv("features/s"+str(0)+"Hurst.csv")
#yu=fulllabel[0]
#gy=pd.DataFrame(yu)
#Label_list = ["Valance","Arousal","Dominance","Liking"]
#gy.columns=Label_list
#gy
gxy=gg.join(gy)



In [9]:
for i in range(1,32):
    gg=pd.read_csv("features/s"+str(i)+"Hurst.csv")
    yu=fulllabel[i]
    gy=pd.DataFrame(yu)
    Label_list = ["Valance","Arousal","Dominance","Liking"]
    gy.columns=Label_list
    nxy=gg.join(gy)
    Dataxy=gxy.append(nxy)
    gxy=Dataxy
Dataxy    

senHurst0  senHurst1  senHurst2  senHurst3  senHurst4  senHurst5  \
0    0.207357   0.206126   0.210456   0.221277   0.284442   0.214171   
1    0.161172   0.231657   0.275299   0.204313   0.258757   0.294660   
2    0.166113   0.264642   0.233001   0.237042   0.256414   0.227641   
3    0.154886   0.192576   0.206195   0.242245   0.232567   0.227261   
4    0.113650   0.115400   0.129648   0.136296   0.162400   0.172677   
5    0.153600   0.152827   0.140391   0.127865   0.203257   0.182056   
6    0.195351   0.228620   0.242343   0.257643   0.242692   0.244664   
7    0.189605   0.204280   0.203644   0.222748   0.236824   0.202516   
8    0.192040   0.284095   0.309022   0.265816   0.282325   0.289760   
9    0.271089   0.318587   0.351095   0.289635   0.285755   0.335088   
10   0.372042   0.419425   0.404088   0.386452   0.351613   0.371335   
11   0.091180   0.093955   0.097766   0.113268   0.144690   0.110109   
12   0.148876   0.188630   0.212470   0.217041   0.208241   0.216558   
13   0.328399   0.233448   0.227244   0.202297   0.143171   0.279467   
14   0.236771   0.311176   0.291701   0.157147   0.260860   0.271980   
15   0.157187   0.164833   0.179504   0.189991   0.227760   0.177238   
16   0.210601   0.200569   0.219848   0.213667   0.191422   0.241050   
17   0.207053   0.207335   0.243745   0.174078   0.208499   0.256980   
18   0.157807   0.187944   0.170539   0.157553   0.187461   0.179426   
19   0.259932   0.264293   0.223905   0.122583   0.210074   0.197382   
20   0.124022   0.141519   0.163303   0.154586   0.258314   0.209967   
21   0.131572   0.167687   0.170264   0.157126   0.201386   0.195841   
22   0.165231   0.174082   0.201809   0.216585   0.195903   0.210254   
23   0.205097   0.224552   0.254725   0.281986   0.354413   0.275758   
24   0.179593   0.173391   0.170934   0.212790   0.199897   0.138288   
25   0.246811   0.289187   0.239986   0.158678   0.190174   0.214498   
26   0.226425   0.138351   0.106125   0.076808   0.122220   0.157665   
27   0.217956   0.291568   0.339915   0.251754   0.287098   0.224567   
28   0.151679   0.161162   0.170824   0.179829   0.167028   0.145040   
29   0.212488   0.209723   0.189718   0.284819   0.229001   0.232031   
..        ...        ...        ...        ...        ...        ...   
10   0.271029   0.296492   0.396193   0.277077   0.300668   0.274230   
11   0.084544   0.297378   0.289759   0.288177   0.354091   0.245700   
12   0.121832   0.439320   0.320035   0.391216   0.450266   0.305769   
13   0.208646   0.314265   0.287288   0.354571   0.238183   0.174781   
14   0.110716   0.190260   0.301076   0.252548   0.187356   0.172744   
15   0.253084   0.535429   0.224805   0.454252   0.589047   0.459930   
16   0.154969   0.277294   0.250833   0.303599   0.301329   0.244313   
17   0.094745   0.213256   0.376810   0.348784   0.226515   0.230370   
18   0.138636   0.270847   0.226075   0.186063   0.268097   0.203899   
19   0.157107   0.230767   0.317205   0.265420   0.259553   0.200554   
20   0.133279   0.260850   0.296049   0.238749   0.303981   0.255267   
21   0.166626   0.328695   0.264286   0.306103   0.384624   0.299279   
22   0.338811   0.273976   0.355791   0.199274   0.230646   0.245911   
23   0.092438   0.184624   0.275194   0.194291   0.195308   0.180726   
24   0.115418   0.235365   0.333007   0.244245   0.216646   0.234131   
25   0.140943   0.287331   0.317464   0.242303   0.277245   0.206536   
26   0.163868   0.221565   0.387605   0.206791   0.251599   0.181905   
27   0.076904   0.296878   0.247789   0.307126   0.310713   0.213338   
28   0.209248   0.221448   0.311068   0.187260   0.214698   0.197905   
29   0.180529   0.184521   0.200846   0.186746   0.193356   0.202236   
30   0.193733   0.195049   0.327396   0.262141   0.192369   0.217033   
31   0.201519   0.321145   0.210317   0.305809   0.246598   0.257838   
32   0.119128   0.180638   0.224744   0.183117   0.192745   0.169696   
33   0.114258   0.373425   0.134498   0.3186

In [12]:
#Dataxy.head(50)


In [25]:
Dataxy=Dataxy.reset_index(drop=True)
Dataxy.to_csv("Dataxy.csv", index=False)

In [26]:
DataAr=Dataxy.iloc[:,:-3]

In [27]:
DataAr

senHurst0  senHurst1  senHurst2  senHurst3  senHurst4  senHurst5  \
0      0.207357   0.206126   0.210456   0.221277   0.284442   0.214171   
1      0.161172   0.231657   0.275299   0.204313   0.258757   0.294660   
2      0.166113   0.264642   0.233001   0.237042   0.256414   0.227641   
3      0.154886   0.192576   0.206195   0.242245   0.232567   0.227261   
4      0.113650   0.115400   0.129648   0.136296   0.162400   0.172677   
5      0.153600   0.152827   0.140391   0.127865   0.203257   0.182056   
6      0.195351   0.228620   0.242343   0.257643   0.242692   0.244664   
7      0.189605   0.204280   0.203644   0.222748   0.236824   0.202516   
8      0.192040   0.284095   0.309022   0.265816   0.282325   0.289760   
9      0.271089   0.318587   0.351095   0.289635   0.285755   0.335088   
10     0.372042   0.419425   0.404088   0.386452   0.351613   0.371335   
11     0.091180   0.093955   0.097766   0.113268   0.144690   0.110109   
12     0.148876   0.188630   0.212470   0.217041   0.208241   0.216558   
13     0.328399   0.233448   0.227244   0.202297   0.143171   0.279467   
14     0.236771   0.311176   0.291701   0.157147   0.260860   0.271980   
15     0.157187   0.164833   0.179504   0.189991   0.227760   0.177238   
16     0.210601   0.200569   0.219848   0.213667   0.191422   0.241050   
17     0.207053   0.207335   0.243745   0.174078   0.208499   0.256980   
18     0.157807   0.187944   0.170539   0.157553   0.187461   0.179426   
19     0.259932   0.264293   0.223905   0.122583   0.210074   0.197382   
20     0.124022   0.141519   0.163303   0.154586   0.258314   0.209967   
21     0.131572   0.167687   0.170264   0.157126   0.201386   0.195841   
22     0.165231   0.174082   0.201809   0.216585   0.195903   0.210254   
23     0.205097   0.224552   0.254725   0.281986   0.354413   0.275758   
24     0.179593   0.173391   0.170934   0.212790   0.199897   0.138288   
25     0.246811   0.289187   0.239986   0.158678   0.190174   0.214498   
26     0.226425   0.138351   0.106125   0.076808   0.122220   0.157665   
27     0.217956   0.291568   0.339915   0.251754   0.287098   0.224567   
28     0.151679   0.161162   0.170824   0.179829   0.167028   0.145040   
29     0.212488   0.209723   0.189718   0.284819   0.229001   0.232031   
...         ...        ...        ...        ...        ...        ...   
1250   0.271029   0.296492   0.396193   0.277077   0.300668   0.274230   
1251   0.084544   0.297378   0.289759   0.288177   0.354091   0.245700   
1252   0.121832   0.439320   0.320035   0.391216   0.450266   0.305769   
1253   0.208646   0.314265   0.287288   0.354571   0.238183   0.174781   
1254   0.110716   0.190260   0.301076   0.252548   0.187356   0.172744   
1255   0.253084   0.535429   0.224805   0.454252   0.589047   0.459930   
1256   0.154969   0.277294   0.250833   0.303599   0.301329   0.244313   
1257   0.094745   0.213256   0.376810   0.348784   0.226515   0.230370   
1258   0.138636   0.270847   0.226075   0.186063   0.268097   0.203899   
1259   0.157107   0.230767   0.317205   0.265420   0.259553   0.200554   
1260   0.133279   0.260850   0.296049   0.238749   0.303981   0.255267   
1261   0.166626   0.328695   0.264286   0.306103   0.384624   0.299279   
1262   0.338811   0.273976   0.355791   0.199274   0.230646   0.245911   
1263   0.092438   0.184624   0.275194   0.194291   0.195308   0.180726   
1264   0.115418   0.235365   0.333007   0.244245   0.216646   0.234131   
1265   0.140943   0.287331   0.317464   0.242303   0.277245   0.206536   
1266   0.163868   0.221565   0.387605   0.206791   0.251599   0.181905   
1267   0.076904   0.296878   0.247789   0.307126   0.310713   0.213338   
1268   0.209248   0.221448   0.311068   0.187260   0.214698   0.197905   
1269   0.180529   0.184521   0.200846   0.186746   0.193356   0.202236   
1270   0.193733   0.195049   0.327396   0.262141   0.192369   0.217033   
1271   0.201519   0.321145   0.210317   0.305809   0.246598   0.257838   
1272   0.1

In [28]:
DataAr=DataAr.replace(np.nan,0)

In [29]:
DataAr

senHurst0  senHurst1  senHurst2  senHurst3  senHurst4  senHurst5  \
0      0.207357   0.206126   0.210456   0.221277   0.284442   0.214171   
1      0.161172   0.231657   0.275299   0.204313   0.258757   0.294660   
2      0.166113   0.264642   0.233001   0.237042   0.256414   0.227641   
3      0.154886   0.192576   0.206195   0.242245   0.232567   0.227261   
4      0.113650   0.115400   0.129648   0.136296   0.162400   0.172677   
5      0.153600   0.152827   0.140391   0.127865   0.203257   0.182056   
6      0.195351   0.228620   0.242343   0.257643   0.242692   0.244664   
7      0.189605   0.204280   0.203644   0.222748   0.236824   0.202516   
8      0.192040   0.284095   0.309022   0.265816   0.282325   0.289760   
9      0.271089   0.318587   0.351095   0.289635   0.285755   0.335088   
10     0.372042   0.419425   0.404088   0.386452   0.351613   0.371335   
11     0.091180   0.093955   0.097766   0.113268   0.144690   0.110109   
12     0.148876   0.188630   0.212470   0.217041   0.208241   0.216558   
13     0.328399   0.233448   0.227244   0.202297   0.143171   0.279467   
14     0.236771   0.311176   0.291701   0.157147   0.260860   0.271980   
15     0.157187   0.164833   0.179504   0.189991   0.227760   0.177238   
16     0.210601   0.200569   0.219848   0.213667   0.191422   0.241050   
17     0.207053   0.207335   0.243745   0.174078   0.208499   0.256980   
18     0.157807   0.187944   0.170539   0.157553   0.187461   0.179426   
19     0.259932   0.264293   0.223905   0.122583   0.210074   0.197382   
20     0.124022   0.141519   0.163303   0.154586   0.258314   0.209967   
21     0.131572   0.167687   0.170264   0.157126   0.201386   0.195841   
22     0.165231   0.174082   0.201809   0.216585   0.195903   0.210254   
23     0.205097   0.224552   0.254725   0.281986   0.354413   0.275758   
24     0.179593   0.173391   0.170934   0.212790   0.199897   0.138288   
25     0.246811   0.289187   0.239986   0.158678   0.190174   0.214498   
26     0.226425   0.138351   0.106125   0.076808   0.122220   0.157665   
27     0.217956   0.291568   0.339915   0.251754   0.287098   0.224567   
28     0.151679   0.161162   0.170824   0.179829   0.167028   0.145040   
29     0.212488   0.209723   0.189718   0.284819   0.229001   0.232031   
...         ...        ...        ...        ...        ...        ...   
1250   0.271029   0.296492   0.396193   0.277077   0.300668   0.274230   
1251   0.084544   0.297378   0.289759   0.288177   0.354091   0.245700   
1252   0.121832   0.439320   0.320035   0.391216   0.450266   0.305769   
1253   0.208646   0.314265   0.287288   0.354571   0.238183   0.174781   
1254   0.110716   0.190260   0.301076   0.252548   0.187356   0.172744   
1255   0.253084   0.535429   0.224805   0.454252   0.589047   0.459930   
1256   0.154969   0.277294   0.250833   0.303599   0.301329   0.244313   
1257   0.094745   0.213256   0.376810   0.348784   0.226515   0.230370   
1258   0.138636   0.270847   0.226075   0.186063   0.268097   0.203899   
1259   0.157107   0.230767   0.317205   0.265420   0.259553   0.200554   
1260   0.133279   0.260850   0.296049   0.238749   0.303981   0.255267   
1261   0.166626   0.328695   0.264286   0.306103   0.384624   0.299279   
1262   0.338811   0.273976   0.355791   0.199274   0.230646   0.245911   
1263   0.092438   0.184624   0.275194   0.194291   0.195308   0.180726   
1264   0.115418   0.235365   0.333007   0.244245   0.216646   0.234131   
1265   0.140943   0.287331   0.317464   0.242303   0.277245   0.206536   
1266   0.163868   0.221565   0.387605   0.206791   0.251599   0.181905   
1267   0.076904   0.296878   0.247789   0.307126   0.310713   0.213338   
1268   0.209248   0.221448   0.311068   0.187260   0.214698   0.197905   
1269   0.180529   0.184521   0.200846   0.186746   0.193356   0.202236   
1270   0.193733   0.195049   0.327396   0.262141   0.192369   0.217033   
1271   0.201519   0.321145   0.210317   0.305809   0.246598   0.257838   
1272   0.1

In [51]:
Xmama = DataAr.iloc[:,:-1]
#print(Xmama)
Ymama = DataAr.iloc[:,40].values
print(Ymama)
#np.shape(Xmama)
#np.shape(Ymama)
from sklearn.model_selection import train_test_split 
from sklearn import tree 
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()

x_train,x_test,y_train,y_test = train_test_split(Xmama,Ymama,test_size=0.25,random_state=0)

regressor.fit(x_train,x_test)

y_pred=regressor.predict(x_test)








import numpy as np
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))






[8.13 4.99 8.05 ... 3.05 3.99 7.15]


ValueError: Found input variables with inconsistent numbers of samples: [960, 320]

In [15]:
gg=pd.read_csv("features/s"+str(0)+"Dfa.csv")
#yu=fulllabel[0]
#gy=pd.DataFrame(yu)
#Label_list = ["Valance","Arousal","Dominance","Liking"]
#gy.columns=Label_list
#gy
#gxy=gg.join(gy)
gxy=gg


In [16]:
for i in range(1,32):
    gg=pd.read_csv("features/s"+str(i)+"Dfa.csv")
#     yu=fulllabel[i]
#     gy=pd.DataFrame(yu)
#     Label_list = ["Valance","Arousal","Dominance","Liking"]
#     gy.columns=Label_list
#     nxy=gg.join(gy)
    nxy=gg
    Dataxy=gxy.append(nxy)
    gxy=Dataxy
Dataxy
Dataxy=Dataxy.reset_index(drop=True)
Dataxy.to_csv("DataDfaxy.csv", index=False)

In [2]:
import pandas as pd 
mama=pd.read_csv("DataDfaxy.csv")
mama2=pd.read_csv("DataHJxy.csv")
mama3=pd.read_csv("DataPfdxy.csv")
mamalast=pd.read_csv("Dataxy.csv")

In [29]:
mama3.head()


senPfd0   senPfd1   senPfd2   senPfd3   senPfd4   senPfd5   senPfd6  \
0  0.559845  0.560418  0.560082  0.559143  0.557953  0.558753  0.557930   
1  0.558684  0.561112  0.561235  0.552965  0.558053  0.560082  0.558741   
2  0.559492  0.560830  0.560345  0.553925  0.557478  0.560094  0.559074   
3  0.559916  0.560757  0.560261  0.558176  0.557908  0.558947  0.558423   
4  0.559539  0.560178  0.559621  0.557753  0.556497  0.558333  0.557886   

    senPfd7   senPfd8   senPfd9    ...     senPfd30  senPfd31  senPfd32  \
0  0.552918  0.559097  0.560696    ...     0.561948  0.562873  0.543607   
1  0.553155  0.558947  0.560916    ...     0.562075  0.562152  0.546172   
2  0.552441  0.558833  0.560575    ...     0.562769  0.561747  0.545122   
3  0.551544  0.559656  0.560611    ...     0.561911  0.563187  0.544739   
4  0.550326  0.558798  0.559940    ...     0.561911  0.562730  0.543223   

   senPfd33  senPfd34  senPfd35  senPfd36  senPfd37  senPfd38  senPfd39  
0  0.540420  0.541963  0.550067       1.0  0.545384  0.650867  0.639130  
1  0.541338  0.542371  0.549163       1.0  0.545909  0.660526  0.638904  
2  0.540457  0.542186  0.550491       1.0  0.545275  0.657582  0.640864  
3  0.541498  0.543968  0.549430       1.0  0.545627  0.646566  0.635666  
4  0.541433  0.544178  0.549296       1.0  0.544532  0.643551  0.635666  

[5 rows x 40 columns]

In [32]:
mamaNew=mama.join(mama3)

In [34]:
mamaNew=mamaNew.join(mamalast)

In [35]:
mamaNew.head()

senDfa0   senDfa1   senDfa2   senDfa3   senDfa4   senDfa5   senDfa6  \
0 -0.002133 -0.000325 -0.000156  0.001114  0.000636 -0.001559  0.001631   
1 -0.006262 -0.006898 -0.009834 -0.008837 -0.005583 -0.005103 -0.005215   
2 -0.007681 -0.011060 -0.008450 -0.003313 -0.008348 -0.004510 -0.008340   
3 -0.017675 -0.012013 -0.012038 -0.021174 -0.018692 -0.003882 -0.013477   
4 -0.009453 -0.006495 -0.006437 -0.008890 -0.005594 -0.008346 -0.004880   

    senDfa7   senDfa8   senDfa9   ...    senHurst34  senHurst35  senHurst36  \
0  0.001380 -0.006688 -0.007642   ...      0.939189    1.033011    0.996901   
1 -0.007236 -0.009539 -0.003813   ...      1.046685    1.085901         NaN   
2 -0.003636 -0.008242 -0.010575   ...      1.063166    1.088769         NaN   
3 -0.007505 -0.014245 -0.013097   ...      1.086076    1.157497    0.978142   
4 -0.004362 -0.003013 -0.008143   ...      1.003874    1.174798         NaN   

   senHurst37  senHurst38  senHurst39  Valance  Arousal  Dominance  Liking  
0    0.809074    0.970004         NaN     7.71     7.60       6.90    7.83  
1    0.936995    0.937595         NaN     8.10     7.31       7.28    8.47  
2    0.543609    0.995391         NaN     8.58     7.54       9.00    7.08  
3    0.735437    1.031969         NaN     4.94     6.01       6.12    8.06  
4    0.831288    1.086271         NaN     6.96     3.92       7.19    6.05  

[5 rows x 124 columns]

In [38]:
mamaNew.to_csv("FinalDataset.csv", index=False)